In [1]:
# word2vec 설명 문서
# https://radimrehurek.com/gensim/models/word2vec.html

# 5.12(화) 수업 참조 site - 챗봇을 위한 seq2seq
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
# http://www.manythings.org/anki/
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"Q:\Python_Data\dataset\fra-eng\fra.txt", sep="\t", header=None)
df

del df[2]
df.columns = ['eng', 'fra']
df

,eng,fra
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
...,...,...
175618,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175619,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175620,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175621,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [10]:
df = df[0:70000]
df.head(10)

,eng,fra
0,Go.,\t Va ! \n
1,Hi.,\t Salut ! \n
2,Hi.,\t Salut. \n
3,Run!,\t Cours ! \n
4,Run!,\t Courez ! \n
5,Who?,\t Qui ? \n
6,Wow!,\t Ça alors ! \n
7,Fire!,\t Au feu ! \n
8,Help!,\t À l'aide ! \n
9,Jump.,\t Saute. \n


In [11]:
# fra 열 전체에 대해 시그널 추가 작업 수행
# start : "\t ", stop: " \n"
# ex) Rattrape-le. -> \t Rattrape-le. \n

df['fra'] = df['fra'].apply(lambda x: "\t " + x + " \n")
df

,eng,fra
0,Go.,\t \t Va ! \n \n
1,Hi.,\t \t Salut ! \n \n
2,Hi.,\t \t Salut. \n \n
3,Run!,\t \t Cours ! \n \n
4,Run!,\t \t Courez ! \n \n
...,...,...
69995,I've never ridden a horse.,\t \t Je ne suis jamais monté à cheval. \n \n
69996,I've never ridden a horse.,\t \t Je ne suis jamais monté sur un cheval. \...
69997,I've never seen Tom drunk.,\t \t Je n'ai jamais vu Tom ivre. \n \n
69998,I've never seen a rainbow.,\t \t Je n'ai jamais vu d'arc-en-ciel. \n \n


In [12]:
# 글자 단위 토큰화
engVocab = set()
fraVocab = set()

for line in df.eng:
    for c in line:
        engVocab.add(c)
        
for line in df.fra:
    for c in line:
        fraVocab.add(c)

In [13]:
engVocabSize = len(engVocab) + 1
fraVocabSize = len(fraVocab) + 1
print(engVocabSize)
print(fraVocabSize)

80
106


In [14]:
engVocab = sorted(list(engVocab))
engVocab[50:80]

fraVocab = sorted(list(fraVocab))
fraVocab[50:80]

['Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«']

In [15]:
engToIndex = dict([(c,i+1) for i,c in enumerate(engVocab)])
engToIndex

fraToIndex = dict([(c,i+1) for i,c in enumerate(fraVocab)])
fraToIndex

{'\t': 1,
 '\n': 2,
 ' ': 3,
 '!': 4,
 '"': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'X': 50,
 'Y': 51,
 'Z': 52,
 'a': 53,
 'b': 54,
 'c': 55,
 'd': 56,
 'e': 57,
 'f': 58,
 'g': 59,
 'h': 60,
 'i': 61,
 'j': 62,
 'k': 63,
 'l': 64,
 'm': 65,
 'n': 66,
 'o': 67,
 'p': 68,
 'q': 69,
 'r': 70,
 's': 71,
 't': 72,
 'u': 73,
 'v': 74,
 'w': 75,
 'x': 76,
 'y': 77,
 'z': 78,
 '\xa0': 79,
 '«': 80,
 '»': 81,
 'À': 82,
 'Ç': 83,
 'É': 84,
 'Ê': 85,
 'Ô': 86,
 'à': 87,
 'â': 88,
 'ç': 89,
 'è': 90,
 'é': 91,
 'ê': 92,
 'ë': 93,
 'î': 94,
 'ï': 95,
 'ô': 96,
 'ù': 97,
 'û': 98,
 'œ': 99,
 'С': 100,
 '\

In [16]:
# 영어 문장 -> 정수 인코딩
encoderInput=[]
for li in df.eng:
    
    t=[]
    for c in li:
        
        t.append(engToIndex[c])
        
    encoderInput.append(t)
    
print(encoderInput[:10])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2], [46, 57, 64, 23], [46, 64, 72, 2], [29, 58, 67, 54, 2], [31, 54, 61, 65, 2], [33, 70, 62, 65, 10]]


In [17]:
# 프랑스어 문장 -> 정수 인코딩
decoderInput=[]
for li in df.fra:
    
    t=[]
    for c in li:
        t.append(fraToIndex[c])
        
    decoderInput.append(t)
    
print(decoderInput[:10])

[[1, 3, 1, 3, 48, 53, 3, 4, 3, 2, 3, 2], [1, 3, 1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2, 3, 2], [1, 3, 1, 3, 45, 53, 64, 73, 72, 14, 3, 2, 3, 2], [1, 3, 1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2, 3, 2], [1, 3, 1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2, 3, 2], [1, 3, 1, 3, 43, 73, 61, 3, 26, 3, 2, 3, 2], [1, 3, 1, 3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2, 3, 2], [1, 3, 1, 3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2, 3, 2], [1, 3, 1, 3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2, 3, 2], [1, 3, 1, 3, 45, 53, 73, 72, 57, 14, 3, 2, 3, 2]]


In [18]:
# 프랑스어 문장 출력
decoderFra=[]
for li in df.fra:
    
    t=[]
    i=0
    for c in li:
        
        if i > 0:
            t.append(fraToIndex[c])
            
        i+=1
        
    decoderFra.append(t)
    
print(decoderFra[:10])

[[3, 1, 3, 48, 53, 3, 4, 3, 2, 3, 2], [3, 1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2, 3, 2], [3, 1, 3, 45, 53, 64, 73, 72, 14, 3, 2, 3, 2], [3, 1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2, 3, 2], [3, 1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2, 3, 2], [3, 1, 3, 43, 73, 61, 3, 26, 3, 2, 3, 2], [3, 1, 3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2, 3, 2], [3, 1, 3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2, 3, 2], [3, 1, 3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2, 3, 2], [3, 1, 3, 45, 53, 73, 72, 57, 14, 3, 2, 3, 2]]


In [19]:
decoderInput

[[1, 3, 1, 3, 48, 53, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 45, 53, 64, 73, 72, 14, 3, 2, 3, 2],
 [1, 3, 1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 43, 73, 61, 3, 26, 3, 2, 3, 2],
 [1, 3, 1, 3, 83, 53, 3, 53, 64, 67, 70, 71, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 27, 73, 3, 58, 57, 73, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 82, 3, 64, 9, 53, 61, 56, 57, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 45, 53, 73, 72, 57, 14, 3, 2, 3, 2],
 [1, 3, 1, 3, 83, 53, 3, 71, 73, 58, 58, 61, 72, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 45, 72, 67, 68, 105, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 27, 70, 70, 92, 72, 57, 13, 72, 67, 61, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 27, 72, 72, 57, 66, 56, 71, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 27, 72, 72, 57, 66, 56, 57, 78, 3, 4, 3, 2, 3, 2],
 [1, 3, 1, 3, 42, 67, 73, 70, 71, 73, 61, 71, 14, 3, 2, 3, 2],
 [1, 3, 1, 3, 29, 67, 66, 72, 61, 66, 73, 57, 78, 14, 3, 2, 3, 2],
 [1, 3,

In [20]:
# 문장 -> 정수 변환 -> 패딩
# 패딩 : 각 언어별로 가장 긴 글자로 구성된 문장 길이로 통일

maxEngLen = max([len(li) for li in df.eng]) # 영어문장의 가장 긴 문자의 길이
maxFraLen = max([len(li) for li in df.fra]) 
print(maxEngLen)
print(maxFraLen)

26
80


In [21]:
# 인코더와 디코더의 문자의 길이가 같을 필요는 없음
from keras.preprocessing.sequence import pad_sequences

encoderInput = pad_sequences(encoderInput, maxlen=maxEngLen, padding='post') # post : 뒷쪽
decoderInput = pad_sequences(decoderInput, maxlen=maxFraLen, padding='post')
decoderFra = pad_sequences(decoderFra, maxlen=maxFraLen, padding='post')

Using TensorFlow backend.


In [22]:
np.shape(encoderInput)
np.shape(decoderInput)

(70000, 80)

In [23]:
from keras.utils import to_categorical

encoderInput = to_categorical(encoderInput)
decoderInput = to_categorical(decoderInput)
decoderFra = to_categorical(decoderFra)

In [24]:
np.shape(encoderInput) # (70000, 26, 80) (전체 문장수, 전체 줄에서 최대 글자수, 글자의 종류)
np.shape(decoderInput) # (70000, 76, 106) 

(70000, 80, 106)

In [25]:
# 트레이닝시 이전 상태의 실제값을 현재 상태의 디코더 입력으로 해야함(예측값으로 하면 안됨)
from keras.layers import Input, Embedding, Dense, LSTM
from keras.models import Model

In [26]:
# ex) ip = Input(shape=(50,1)) -> feature : 1개, time-step(시점) : 50개
# it = LSTM(출력)(ip)
# d1 = Dense(10, activation='relu')(it)
# d2 = Dense(1, activation='sigmoid')(d1)
# Model(inputs=ip, outputs=d2)

In [27]:
encoderInputs = Input(shape=(None, engVocabSize))
decoderInputs = Input(shape=(None, fraVocabSize)) # 디코더 입력 = Input(None(maxFraLen), 프랑스어 문자 종류 수)

# 인코더 LSTM 셀
encoderLSTM = LSTM(units=256, return_state=True) # return_state=True : 인코더의 마지막 상태 정보를 디코더의 입력 상태 정보로 전달 옵션

# 디코더 LSTM 셀
decoderLSTM = LSTM(units=256, return_sequences=True, return_state=True)

# 인코더 LSTM셀의 입력 정의
_, stateH, stateC = encoderLSTM(encoderInputs) # _, 히든상태(위), 셀상태(오른쪽)
encoderStates = [stateH, stateC] # 컨텍스트 벡터

decoderOutputs, _, _ = decoderLSTM(decoderInputs, initial_state=encoderStates)
decoderSoftmax = Dense(fraVocabSize, activation="softmax")
decoderOutputs = decoderSoftmax(decoderOutputs)

In [28]:
model = Model(inputs=[encoderInputs,decoderInputs], outputs=decoderOutputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 80)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 106)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 345088      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  371712      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [31]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit(x=[encoderInput,decoderInput], y=decoderFra, batch_size=256, epochs=10, validation_split=0.2)

Train on 56000 samples, validate on 14000 samples
Epoch 1/10
56000/56000 [==============================] - 63s 1ms/step - loss: 0.6477 - val_loss: 0.7377
Epoch 2/10
 1536/56000 [..............................] - ETA: 53s - loss: 0.5792

KeyboardInterrupt: 